In [1]:
import requests 
import pymysql
try:
    con = pymysql.connect(host='192.168.56.101', 
                          user='encore', password='123', 
                          charset='utf8', db='test') #MySQL Connection 연결
    ''' 
    host : blah blah~ 
    return cur, con
    '''
    cur = con.cursor() #Connection으로부터 Cursor 생성
except Exception as e:
    print ("error ->", e)


In [19]:
krx_url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
payload = {"bld"  : "dbms/MDC/STAT/standard/MDCSTAT01901",
            "locale"  : "ko_KR",
            "mktId"  : "ALL",
            "share"  : "1",
            "csvxls_isNo"  : "false",}
r = requests.post(krx_url, data=payload) #데이터 크롤링
rt = r.json()


In [21]:
sql = "INSERT INTO KRX VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
for data in rt['OutBlock_1']:
    try:
        cur.execute(sql, list(data.values()))
    except:
        pass

con.commit()     #SQL을 실행해서 KRX의 값을 리스트로 넣어줘


In [6]:

import pandas as pd #판다스를 실행해서 SQL의 KRX 쿼리 값을 불러와
krx = pd.read_sql_query("""SELECT * 
                        FROM KRX
                        WHERE MKT_TP_NM IN ('KOSPI', 'KOSDAQ')""", con)


c:\Users\Playdata\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
krx.loc[1, 'ISU_CD'] # 라벨 값으로 데이터 접근, ISU_CD의 1번 인덱스 값


'KR7000040006'

In [8]:
krx.loc[1:, ['ISU_CD', 'ISU_NM', 'ISU_SRT_CD']] # 쿼리의 1번부터 끝까지 값


,ISU_CD,ISU_NM,ISU_SRT_CD
1,KR7000040006,KR모터스보통주,000040
2,KR7000050005,경방보통주,000050
3,KR7000070003,삼양홀딩스보통주,000070
4,KR7000071001,삼양홀딩스1우선주,000075
5,KR7000080002,하이트진로보통주,000080
...,...,...,...
2510,KR8392080006,제이티씨,950170
2511,KR8344390008,미투젠,950190
2512,KR8840150005,소마젠,950200
2513,KR8702070002,프레스티지바이오파마KDR,950210


In [9]:
krx.iloc[0:, [1,2,3]]  #0번부터 끝까지 1,2,3번의 쿼리를 가져와줘


,ISU_SRT_CD,ISU_NM,ISU_ABBRV
0,000020,동화약품보통주,동화약품
1,000040,KR모터스보통주,KR모터스
2,000050,경방보통주,경방
3,000070,삼양홀딩스보통주,삼양홀딩스
4,000075,삼양홀딩스1우선주,삼양홀딩스우
...,...,...,...
2510,950170,제이티씨,JTC
2511,950190,미투젠,미투젠
2512,950200,소마젠,소마젠
2513,950210,프레스티지바이오파마KDR,프레스티지바이오파마


In [10]:
import requests
import pandas as pd 

def get_stock(code, page):
    naver_url = "https://finance.naver.com/item/sise_day.naver?code={}&page={}"
    head = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}
    r= requests.get(naver_url.format(code,page),headers=head)
    return pd.read_html(r.text)[0].dropna()


In [11]:
if __name__=="__main__":
    print("haha")


haha


In [12]:
import naver
from naver import get_stock
import numpy as np

samsung = get_stock("005930", 2)


In [13]:
samsung.columns


Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량'], dtype='object')

In [14]:
samsung[['종가', '전일비', '시가', '고가', '저가', '거래량']] = \
samsung[['종가', '전일비', '시가', '고가', '저가', '거래량']].astype(np.int64)
samsung['거래금액'] = samsung['종가'] * samsung['거래량']


In [17]:
# 종목 -> symbol
# 날짜 -> stock_date
# 종가 -> close
# 전일비 -> before
# 시가 -> open
# 고가 -> high
# 저가 -> low
# 거래량 -> vol
# 거래금액 -> money


In [15]:
samsung['종목'] = '005930'
samsung = samsung[['종목', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '거래금액', ]]


In [16]:
sql = "INSERT INTO stock_day VALUES (%s , %s , %s , %s , %s , %s , %s , %s , %s)"

for idx, row in samsung.iterrows():
    try:
        cur.execute(sql,list(row.values))
    except:
        pass
    
con.commit()


In [18]:
import pymysql

def connection(host='192.168.56.101', user='encore', password='123', 
                              charset='utf8', db='test'):
    try:
        con = pymysql.connect(host=host, 
                              user=user, password=password ,
                              charset=charset, db=db)
        cur = con.cursor()
    except Exception as e:
        print ("error ->", e)
    
    return cur, con
